In [49]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from random import sample
from collections import Counter


In [5]:
def readCsv(fileName):
    fullFileName = r'C:\Users\Owner\McGill\4thYear\COMP551\Assignments\Assignment2\Datasets\DS' + fileName
    df = pd.read_csv(fullFileName, encoding='utf-8', header = None,
                 comment='#', sep=',')
    
    return df

In [55]:
def kMM():
    dfTrain = readCsv('1_train.csv')
    dfTest = readCsv('1_test.csv')
    distancesList = list()
    
    for i in range(0, len(dfTrain)):
        distancesList.append(findDistance(dfTest.iloc[0,:], dfTrain.iloc[i, :]))
    
#     distancesList.sort()
#     print(distancesList)
    return sorted(distancesList)

kMM()

[(4.0229305547264378, 0.0),
 (4.4051872882255676, 1.0),
 (4.4452545247496458, 1.0),
 (4.6053422068490306, 0.0),
 (4.6352445747256255, 1.0),
 (4.6868969353202603, 0.0),
 (4.8463787012606003, 1.0),
 (4.9102332162263727, 0.0),
 (4.948608694366297, 1.0),
 (4.9643516635805574, 0.0),
 (5.1309995135513935, 0.0),
 (5.1351045712949288, 0.0),
 (5.237072335153969, 0.0),
 (5.3490994189326466, 0.0),
 (5.3659227425039191, 0.0),
 (5.3674674250864252, 1.0),
 (5.4009339730163184, 1.0),
 (5.4081612012997917, 0.0),
 (5.4138008665077164, 1.0),
 (5.4562936238482846, 1.0),
 (5.4727341666821694, 1.0),
 (5.4846230485517573, 1.0),
 (5.487928550483324, 0.0),
 (5.5010707338221536, 1.0),
 (5.5347292053675599, 0.0),
 (5.5727295125543996, 1.0),
 (5.572932720381897, 1.0),
 (5.5980509437362285, 0.0),
 (5.6272279531008422, 0.0),
 (5.6384847541447956, 0.0),
 (5.6446412248454676, 0.0),
 (5.6578234622391115, 0.0),
 (5.6646808095306707, 1.0),
 (5.6667231765840667, 0.0),
 (5.6698878527013834, 0.0),
 (5.6978555859611282, 0.

In [67]:
def vote(neighbors):
    classCounter = Counter()
    for neighbor in neighbors:
        classCounter[neighbor[1]]+=1
#         print(1)
    return classCounter

In [52]:
def findDistance(vectorTest, vectorTrain):
    vectorTrainClass = vectorTrain[20]
    vectorTest = vectorTest[0:20].as_matrix()
    vectorTrain = vectorTrain[0:20].as_matrix()
    distance = np.linalg.norm(vectorTest-vectorTrain)
    return (distance, vectorTrainClass)
    
    

In [79]:
def main():
    listSorted = kMM()
#     num = 0
#     for x in listSorted:
#         print(x[1])
#     print(num)
    classCounter = vote(listSorted[0:20])
    print(classCounter.most_common(2))
    
main()

[(0.0, 11), (1.0, 9)]
